In [ ]:
!python main.py

In [ ]:
from scholar.post_maker import make_all
make_all()

# Archive

## Add new column called arxiv_term in SQLite

In [ ]:
import sqlite3

# Reset the new columns
# with sqlite3.connect('scholar/paper.db') as conn:
#     c = conn.cursor()
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_short;')
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_term_long;')
#     conn.commit()

with sqlite3.connect('scholar/paper.db') as conn:
    c = conn.cursor()
    c.execute('ALTER TABLE paper ADD COLUMN arxiv_group_tag VARCHAR(255);')
    c.execute('ALTER TABLE paper ADD COLUMN arxiv_category_tag VARCHAR(255);')
    conn.commit()


## Patch category and group tag to database

In [ ]:
from time import sleep
from scholar.database import Paper
from scholar.api import ARXIV_GROUP, ARXIV_CATEGORY, get_arxiv_category
p = Paper.select().where(Paper.arxiv_group_tag == None)

for paper in p.iterator():
    print(paper.title)
    try:
        category_tag=get_arxiv_category(paper.title)
    except AttributeError:
        print(f'{paper.title} not found.')
        continue

    # Make group tag and push to database
    group_tag = category_tag.split('.')[0]
    if group_tag in ARXIV_GROUP:
        paper.arxiv_group_tag=group_tag
    
    if category_tag in ARXIV_CATEGORY:
        paper.arxiv_category_tag=category_tag
    paper.save()
    sleep(0.5)  # arxiv.org has a rate limit of 3 requests per second